In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import h5py
import json
import plot_SNVM_data as pSd
from scipy.optimize import curve_fit

In [ ]:
def exponential(t,A0,tau,t0,C):
    return A0*np.exp(-(t-t0)/tau)+C
def exponentialNot0(t,A0,tau,C):
    return A0*np.exp(-(t)/tau)+C
#def odrexp(B,t):
#    return B[0]*np.exp(-(t-B[2])/B[1])+B[3]
def odrexp(B,t):
    return B[0]*np.exp(-(t)/B[1])+B[2]
def twotopow(t,A0,tau,t0,C):
    return A0*np.power(2,(-(t-t0)/tau))+C
def tentopow(t,A0,tau,t0,C):
    return A0*np.power(10,(-(t-t0)/tau))+C
def fiftytopow(t,A0,tau,t0,C):
    return A0*np.power(50,(-(t-t0)/tau))+C
def logofexpNot0(t,A0,tau,C):
    return np.log(A0*np.exp(-(t)/tau)+C)
def logofexp(t,A0,tau,t0,C):
    return np.log(A0*np.exp(-(t-t0)/tau)+C)
def noCexp(t,A0,tau):
    return(A0*np.exp(-t/tau))
def gaussian_pdf(x,sigma,mu):
    return(1/(sigma*np.sqrt(2*np.pi))*np.exp((x-mu)**2/(2*sigma**2)))
def gaussian(x,x0,a,b):
    return(a*np.exp((x-x0)**2/(2*b**2)))
def poisson(x,lam,a,x0):
    return(a*np.power(lam,(x-x0))/scipy.special.factorial(x-x0)*np.exp(-lam))
def poissonNoFact(x,lam,a,x0):
    return(a*np.power(lam,(x-x0))*np.exp(-lam))

In [ ]:
# List all items in basepath
basepath = '/home/julisko/Seafile/My Library/SNVM/QSM_data/Pulse experiments'
basepath_folders = []
basepath_foldernames = []

for i in range(len(os.listdir(basepath))):
    if os.path.isdir(basepath+'/'+sorted(os.listdir(basepath))[i]):
        basepath_folders.append(basepath+'/'+sorted(os.listdir(basepath))[i])
        basepath_foldernames.append(sorted(os.listdir(basepath))[i])
print(basepath_foldernames)

Pulse_exp_dayfolders = basepath_foldernames


In [ ]:
# read in all data

Au25filepath = '/home/julisko/Seafile/My Library/SNVM/QSM_data/Measurement Data/Pulse experiments/2024-05-23/relaxation_laser/2024-05-23-17-09-43-relaxation_laser/'
Au25file, Au25metadata = h5py.File(Au25filepath+'seq0/eval/relaxation.h5'), json.load(open(Au25filepath + 'imageMeta.json'))

Au45filepath = '/home/julisko/Seafile/My Library/SNVM/QSM_data/Measurement Data/Pulse experiments/2024-05-23/relaxation_laser/2024-05-23-01-18-18-relaxation_laser/'
Au45file, Au45metadata = h5py.File(Au45filepath+'seq0/eval/relaxation.h5'), json.load(open(Au45filepath + 'imageMeta.json'))

Au65filepath = '/home/julisko/Seafile/My Library/SNVM/QSM_data/Measurement Data/Pulse experiments/2024-05-23/relaxation_laser/2024-05-23-13-13-42-relaxation_laser/'
Au65file, Au65metadata = h5py.File(Au65filepath+'seq0/eval/relaxation.h5'), json.load(open(Au65filepath + 'imageMeta.json'))

AuDisenfilepath = '/home/julisko/Seafile/My Library/SNVM/QSM_data/Measurement Data/Pulse experiments/2024-05-24/relaxation_laser/2024-05-24-12-07-04-relaxation_laser/'
AuDisenfile, AuDisenmetadata = h5py.File(AuDisenfilepath+'seq0/eval/relaxation.h5'), json.load(open(AuDisenfilepath + 'imageMeta.json'))




Glass25filepath = '/home/julisko/Seafile/My Library/SNVM/QSM_data/Measurement Data/Pulse experiments/2024-05-25/relaxation_laser/2024-05-25-12-46-50-relaxation_laser/'
Glass25file, Glass25metadata = h5py.File(Glass25filepath+'seq0/eval/relaxation.h5'), json.load(open(Glass25filepath + 'imageMeta.json'))

Glass45filepath = '/home/julisko/Seafile/My Library/SNVM/QSM_data/Measurement Data/Pulse experiments/2024-05-24/relaxation_laser/2024-05-24-20-10-04-relaxation_laser/'
Glass45file, Glass45metadata = h5py.File(Glass45filepath+'seq0/eval/relaxation.h5'), json.load(open(Glass45filepath + 'imageMeta.json'))

Glass65filepath = '/home/julisko/Seafile/My Library/SNVM/QSM_data/Measurement Data/Pulse experiments/2024-05-26/relaxation_laser/2024-05-26-22-03-11-relaxation_laser/'
Glass65file, Glass65metadata = h5py.File(Glass65filepath+'seq0/eval/relaxation.h5'), json.load(open(Glass65filepath + 'imageMeta.json'))

GlassDisenfilepath = '/home/julisko/Seafile/My Library/SNVM/QSM_data/Measurement Data/Pulse experiments/2024-05-25/relaxation_laser/2024-05-25-02-01-24-relaxation_laser/'
GlassDisenfile, GlassDisenmetadata = h5py.File(GlassDisenfilepath+'seq0/eval/relaxation.h5'), json.load(open(GlassDisenfilepath + 'imageMeta.json'))


Aufiles = [Au25file,Au45file,Au65file,AuDisenfile]
Aumetadata = [Au25metadata,Au45metadata,Au65metadata,AuDisenmetadata]

Glassfiles = [Glass25file,Glass45file,Glass65file,GlassDisenfile]
Glassmetadata = [Glass25metadata,Glass45metadata,Glass65metadata,GlassDisenmetadata]



In [ ]:
fig, ax = plt.subplots()

refamps = [0.25, 0.45, 0.65, 1]
colors = ['C0','C1','C2','C3']

for (index,h5file) in enumerate(Glassfiles):
    delays = pSd.ExtractData(h5file, datatype = 'delay')[0][0]
    ms0s = pSd.ExtractData(h5file, datatype = 'ms0')[0][0]
    ref0s = pSd.ExtractData(h5file, datatype = 'ref0')[0][0]
    contrasts = pSd.ExtractData(h5file, datatype = 'contrastms0')[0][0]

    p0 = [1/8,1e2,0.9]
    lowerbounds = [-np.inf,0,-np.inf]
    upperbounds = [np.inf,np.inf,np.inf]
    try:
        popt, pcov = curve_fit(exponentialNot0,delays,contrasts, p0=p0,bounds=[lowerbounds,upperbounds])
    except:
        print('error fitting')
        #popt=[0,0,0]

    ax.plot(delays[5:]*1e6,contrasts[5:],'o',label=str(int(100*refamps[index])) + ' % ref. amp.', color = colors[index])
    ax.plot(delays[5:]*1e6,exponentialNot0(delays[5:],*popt),'-', color = colors[index])#,label=r'Fit at ' + str(int(100*refamps[index])) + ' % ref. amp.')#ted $T_1$: ('+str(np.round(popt[1]*1e6,2))+r'$\pm$'+str(np.round(np.sqrt(np.diag(pcov))[1]*1e6,3))+') µs')
    

ax.set_xlabel(r'Delay Time $\tau$ (µs)')
ax.set_ylabel(r'${I_{delay}}/{I_{ref}}$')
plt.rc('font', size=15)
plt.rc('axes', titlesize=15)
#plt.ylim(bottom=0.87)
#ax.set_yticks([])
ax.set_xscale('log')
ax.set_title('Bare Glass')
ax.legend()

#plt.savefig('/home/julisko/Seafile/My Library/SNVM/20230922_QZabre_Visit_Garnets_SMMs_2DMag/20230922_QZabre_Analysis/Relaxometry/reducedT1_both_linscale.png',bbox_inches = 'tight', dpi=300)

plt.show()


In [ ]:
#GlassDyCu525filepath = '/home/julisko/Seafile/My Library/SNVM/QSM_data/Measurement Data/Pulse experiments/2024-05-27/relaxation_laser/2024-05-27-20-36-24-relaxation_laser/'
#GlassDyCu525file, GlassDyCu525metadata = h5py.File(GlassDyCu545filepath+'seq0/eval/relaxation.h5'), json.load(open(GlassDyCu545filepath + 'imageMeta.json'))

GlassDyCu545filepath = '/home/julisko/Seafile/My Library/SNVM/QSM_data/Measurement Data/Pulse experiments/2024-05-28/relaxation_laser/2024-05-28-03-29-09-relaxation_laser/'
GlassDyCu545file, GlassDyCu545metadata = h5py.File(GlassDyCu545filepath+'seq0/eval/relaxation.h5'), json.load(open(GlassDyCu545filepath + 'imageMeta.json'))

#GlassDyCu5Disenfilepath = '/home/julisko/Seafile/My Library/SNVM/QSM_data/Measurement Data/Pulse experiments/2024-05-27/relaxation_laser/2024-05-27-20-36-24-relaxation_laser/'
#GlassDyCu5Disenfile, GlassDyCu5Disenmetadata = h5py.File(GlassDyCu5Disenfilepath+'seq0/eval/relaxation.h5'), json.load(open(GlassDyCu5Disenfilepath + 'imageMeta.json'))

#GlassDyCu5files = [GlassDyCu545file, GlassDyCu5Disenfile]

# compare 45% for bare glass and with DyCu5 molecules

refvsDyCu5files = [Glass45file, GlassDyCu545file]

fig, ax = plt.subplots()

refamps = [0.45]
colors = ['C0','C1']#,'C3']
labels = ['Bare Glass', 'Glass with DyCu5 SMMs']

for (index,h5file) in enumerate(refvsDyCu5files):
    delays = pSd.ExtractData(h5file, datatype = 'delay')[0][0]
    ms0s = pSd.ExtractData(h5file, datatype = 'ms0')[0][0]
    ref0s = pSd.ExtractData(h5file, datatype = 'ref0')[0][0]
    contrasts = pSd.ExtractData(h5file, datatype = 'contrastms0')[0][0]

    p0 = [1/8,1e2,0.9]
    lowerbounds = [-np.inf,0,-np.inf]
    upperbounds = [np.inf,np.inf,np.inf]
    try:
        popt, pcov = curve_fit(exponentialNot0,delays,contrasts, p0=p0,bounds=[lowerbounds,upperbounds])
    except:
        print('error fitting')
        #popt=[0,0,0]

    ax.plot(delays[5:]*1e6,contrasts[5:],'o', label=labels[index], color = colors[index])
    ax.plot(delays[5:]*1e6,exponentialNot0(delays[5:],*popt),'-', color = colors[index])#,label=r'Fit at ' + str(int(100*refamps[index])) + ' % ref. amp.')#ted $T_1$: ('+str(np.round(popt[1]*1e6,2))+r'$\pm$'+str(np.round(np.sqrt(np.diag(pcov))[1]*1e6,3))+') µs')
    

ax.set_xlabel(r'Delay Time $\tau$ (µs)')
ax.set_ylabel(r'${I_{delay}}/{I_{ref}}$')
plt.rc('font', size=15)
plt.rc('axes', titlesize=15)
#plt.ylim(bottom=0.87)
#ax.set_yticks([])
ax.set_xscale('log')
ax.set_title('AFM Feedback 45% Reference Amplitude')
ax.legend()

#plt.savefig('/home/julisko/Seafile/My Library/SNVM/20230922_QZabre_Visit_Garnets_SMMs_2DMag/20230922_QZabre_Analysis/Relaxometry/reducedT1_both_linscale.png',bbox_inches = 'tight', dpi=300)

plt.show()




In [ ]:
GlassDyCu525filepath = '/home/julisko/Seafile/My Library/SNVM/QSM_data/Measurement Data/Pulse experiments/2024-05-28/relaxation_laser/2024-05-28-12-09-11-relaxation_laser/'
GlassDyCu525file, GlassDyCu525metadata = h5py.File(GlassDyCu525filepath+'seq0/eval/relaxation.h5'), json.load(open(GlassDyCu525filepath + 'imageMeta.json'))

#GlassDyCu545filepath = '/home/julisko/Seafile/My Library/SNVM/QSM_data/Measurement Data/Pulse experiments/2024-05-28/relaxation_laser/2024-05-28-03-29-09-relaxation_laser/'
#GlassDyCu545file, GlassDyCu545metadata = h5py.File(GlassDyCu545filepath+'seq0/eval/relaxation.h5'), json.load(open(GlassDyCu545filepath + 'imageMeta.json'))

#GlassDyCu5Disenfilepath = '/home/julisko/Seafile/My Library/SNVM/QSM_data/Measurement Data/Pulse experiments/2024-05-27/relaxation_laser/2024-05-27-20-36-24-relaxation_laser/'
#GlassDyCu5Disenfile, GlassDyCu5Disenmetadata = h5py.File(GlassDyCu5Disenfilepath+'seq0/eval/relaxation.h5'), json.load(open(GlassDyCu5Disenfilepath + 'imageMeta.json'))

#GlassDyCu5files = [GlassDyCu545file, GlassDyCu5Disenfile]

# compare 25% for bare glass and with DyCu5 molecules

refvsDyCu5files = [Glass25file, GlassDyCu525file]

fig, ax = plt.subplots()

refamps = [0.25]
colors = ['C0','C1']#,'C3']
labels = ['Bare Glass', 'Glass with DyCu5 SMMs']

for (index,h5file) in enumerate(refvsDyCu5files):
    delays = pSd.ExtractData(h5file, datatype = 'delay')[0][0]
    ms0s = pSd.ExtractData(h5file, datatype = 'ms0')[0][0]
    ref0s = pSd.ExtractData(h5file, datatype = 'ref0')[0][0]
    contrasts = pSd.ExtractData(h5file, datatype = 'contrastms0')[0][0]

    p0 = [1/8,1e2,0.9]
    lowerbounds = [-np.inf,0,-np.inf]
    upperbounds = [np.inf,np.inf,np.inf]
    try:
        popt, pcov = curve_fit(exponentialNot0,delays,contrasts, p0=p0,bounds=[lowerbounds,upperbounds])
    except:
        print('error fitting')
        #popt=[0,0,0]

    ax.plot(delays[5:]*1e6,contrasts[5:],'o', label=labels[index], color = colors[index])
    ax.plot(delays[5:]*1e6,exponentialNot0(delays[5:],*popt),'-', color = colors[index])#,label=r'Fit at ' + str(int(100*refamps[index])) + ' % ref. amp.')#ted $T_1$: ('+str(np.round(popt[1]*1e6,2))+r'$\pm$'+str(np.round(np.sqrt(np.diag(pcov))[1]*1e6,3))+') µs')
    

ax.set_xlabel(r'Delay Time $\tau$ (µs)')
ax.set_ylabel(r'${I_{delay}}/{I_{ref}}$')
plt.rc('font', size=15)
plt.rc('axes', titlesize=15)
#plt.ylim(bottom=0.87)
#ax.set_yticks([])
#ax.set_xscale('log')
ax.set_title('AFM Feedback 25% Reference Amplitude')
ax.legend()

#plt.savefig('/home/julisko/Seafile/My Library/SNVM/20230922_QZabre_Visit_Garnets_SMMs_2DMag/20230922_QZabre_Analysis/Relaxometry/reducedT1_both_linscale.png',bbox_inches = 'tight', dpi=300)

plt.show()




In [ ]:
RefAmps = [0.25, 0.45, 0.65,1.00]
T1s = [529.4, 716.6, 775.6, 3730.0]
T1errs = [43.6, 47.9, 88.5, 902.1]

RefAmpsRetry = [0.25]
T1sRetry = [702.2]
T1errsRetry = [57.1]

fig, ax = plt.subplots()
ax.set_title('Bare Au')
ax.errorbar(RefAmps, T1s, yerr = T1errs, label='first set')
ax.errorbar(RefAmpsRetry, T1sRetry, yerr = T1errsRetry, label='second set')
ax.set_ylabel(r'$T_1$ $(\mu s)$')
ax.set_xlabel(r'AFM Reference Amplitude (%)')
plt.legend()

In [ ]:
glass_RefAmps = [0.25, 0.45, 0.65, 1.00]
glass_T1s = [4.479, 5.481, 5.0518, 4.692]
glass_T1errs = [0.371, 1.415, 0.5502, 0.596]


glass_fig, glass_ax = plt.subplots()
glass_ax.set_title('Bare Glass Slide')
glass_ax.errorbar(glass_RefAmps, glass_T1s, yerr = glass_T1errs)
glass_ax.set_ylabel(r'$T_1$ $( ms)$')
glass_ax.set_xlabel(r'AFM Reference Amplitude (%)')
plt.legend()